## Numerical Analysis - Fall semester 2024

# Serie 12 - ODEs and systems of ODEs

First importing some packages.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy as sp

<hr style="clear:both">

### Stability of system of ODEs

We consider a system of ordinary differential equations of the form

$$
\begin{cases}
  \frac{\mathrm{d}\mathbf{u}(t)}{\mathrm{d}t} = \mathbf{f}(t, \mathbf{u}(t)), & t \in (0, T],  \\
  \mathbf{u}(0)=\mathbf{u}_0,
\end{cases}
$$

where $\mathbf{u}(t) \in \mathbb{R}^m$ is a vector and $\mathbf{f}: \mathbb{R}^m \to \mathbb{R}^m$ a vector-valued function. We want to approximate $\mathbf{u}(t_n)$ at the discrete time-steps $t_n = n\Delta t$, $n = 0, 1, 2, \ldots,N$, where $\Delta t = T/N$ is the step size and $N$ the number of time-steps.

<div class="alert alert-success">
    
**Exercise 1:** Complete the function `forward_euler_sys` which implements the forward Euler method to compute the approximations $\mathbf{u}^{n} \approx \mathbf{u}(t_n)$ for $n = 0, 1, 2, \dots, N$.

*Hint:* The implementation is almost identical to the function `forward_euler` which you have implemented in last week's notebook, except that now the approximations $\mathbf{u}^{n}$ will be vector-valued, so the input `u_0` will be a NumPy array of size $m$ and the output `u` should be a NumPy array of size $N \times m$.
</div>

In [ ]:
def forward_euler_sys(f, u_0, T, N):
    ### BEGIN SOLUTION
    dt = T / N
    u = np.zeros((N + 1, len(u_0)))
    u[0] = u_0
    for n in range(N):
        u[n + 1] = u[n] + dt * f(n * dt, u[n])
    ### END SOLUTION
    return u

We also provide you with the implementation of the backward Euler method for solving systems of ordinary differential equations. Also this implementation is almost identical to what you have encountered in last week's exercise notebook.

In [ ]:
def backward_euler_sys(f, u_0, T, N):
    dt = T / N
    u = np.zeros((N + 1, len(u_0)))
    u[0] = u_0
    for n in range(N):
        # Function for which we need to find a zero
        def g(x):
            return x - dt * f((n + 1) * dt, x) - u[n]
        # Method which finds the zero of the function g
        u[n + 1] = sp.optimize.root(g, u[n], method="hybr").x
    return u

Given is the linear system of ordinary differential equations
$$
\begin{cases}
  \frac{\mathrm{d}\mathbf{u}(t)}{\mathrm{d}t} = A \mathbf{u}(t) = \mathbf{f}(t, \mathbf{u}(t)), & t \in (0, 20],  \\
  \mathbf{u}(0)=\begin{pmatrix} 1 \\ 0 \end{pmatrix},
\end{cases}
$$

with matrix $A \in \mathbb{R}^{2 \times 2}$ defined as

$$
A = \begin{pmatrix} 0 & 1 \\ -1 & 0 \end{pmatrix}.
$$

The exact solution of this system is

$$
\mathbf{u}(t) = \begin{pmatrix} \cos(t) \\ - \sin(t) \end{pmatrix}
$$

We provide you with the function `plot_solution`, which visualizes the approximate solutions $\boldsymbol{u}^n$ at the time steps $t_n$ for $n=0, 1, 2, \dots, N$. It takes as input the time steps $t_0, t_1, t_2, \dots, t_N$ in a NumPy array `t` of size $N$, and the corresponding solutions $\boldsymbol{u}^0, \boldsymbol{u}^1, \boldsymbol{u}^2, \dots, \boldsymbol{u}^N$ in a NumPy array `u` of size $(N + 1) \times m$.

In [ ]:
def plot_solution(t, u):
    fig, ax = plt.subplots()

    # Determine x- and y-values as well as midpoints
    x = u[:, 0]
    y = u[:, 1]
    x_midpts = np.hstack((x[0], 0.5 * (x[1:] + x[:-1]), x[-1]))
    y_midpts = np.hstack((y[0], 0.5 * (y[1:] + y[:-1]), y[-1]))

    # Define line segments
    coord_start = np.column_stack((x_midpts[:-1], y_midpts[:-1]))[:, np.newaxis, :]
    coord_mid = np.column_stack((x, y))[:, np.newaxis, :]
    coord_end = np.column_stack((x_midpts[1:], y_midpts[1:]))[:, np.newaxis, :]
    segments = np.concatenate((coord_start, coord_mid, coord_end), axis=1)
    linewidths = np.linspace(0, 6, len(t))[::-1]
    lc = mpl.collections.LineCollection(segments, linewidths=linewidths, cmap="plasma", capstyle="butt")
    lc.set_array(t)

    # Plot the line segments
    lines = ax.add_collection(lc)
    fig.colorbar(lines, label=r"parameter value $t$")
    ax.scatter(x[0], y[0], color=mpl.colormaps["plasma"](0), s=70)
    ax.annotate(r"$u_0$", (x[0], y[0]), xytext=(-17, 5), textcoords='offset points')
    ax.margins(0.2, 0.2)
    ax.add_patch(mpl.patches.Circle((0, 0), radius=1, fill=False, edgecolor="black", linestyle="--", linewidth=2))
    plt.xlabel(r"$u_1(t)$")
    plt.ylabel(r"$u_2(t)$")
    plt.grid()
    plt.show()

Below, we visualize the exact solution for $N = 200$ time steps in $[0, 20]$, which we can obtain from the function `u`.

In [ ]:
# Function which returns the exact solutions at the values t
def u(t, u_0):
    return np.array([np.cos(t) * u_0[0] - np.sin(t) * u_0[1], - np.sin(t) * u_0[0] - np.cos(t) * u_0[1]]).T

# Parameters
T = 20
u_0 = np.array([1, 0])
N = 200
t = np.linspace(0, T, N + 1)

# Plots the exact solution
u_exact = u(t, u_0)
plot_solution(t, u_exact)

<div class="alert alert-success">
    
**Exercise 2:** Plot the approximate solutions of the system of ordinary differential equations from the forward and backward Euler methods for $N=200$ time steps. How well do the methods approximate the exact solution?
</div>

In [ ]:
# Right-hand side of the ODE
def f(t, u):
    # Returns the value of the right-hand side of the ODE
    ### BEGIN SOLUTION
    return np.array([u[1], -u[0]])
    ### END SOLUTION

### BEGIN SOLUTION
# Parameters
T = 20
u_0 = np.array([1, 0])
N = 200

# Solve ODEs with the two methods
u_forward_euler = forward_euler_sys(f, u_0, T, N)
u_backward_euler = backward_euler_sys(f, u_0, T, N)

# Visualize the results
plot_solution(t, u_forward_euler)
plot_solution(t, u_backward_euler)

# Both methods diverge from the exact solution:
# The forward Euler method spirals outwards, while the Backward Euler
# spirals inward from the unit circle, on which the exact solution lies.

### END SOLUTION

The Crank-Nicolson method is defined by the iterative formula

$$
\mathbf{u}^{n+1} = \mathbf{u}^{n} + \frac{\Delta t}{2} \left( \mathbf{f}(t_n, \mathbf{u}^{n}) +  \mathbf{f}(t_{n+1}, \mathbf{u}^{n+1}) \right), \quad n=0, 1, 2, \dots
$$

It is an implicit method. As you have seen in the lecture, we will have to find a zero $\boldsymbol{\alpha} \in \mathbb{R}^m$ of the function

$$
\mathbf{g}_n(\mathbf{x}) = \mathbf{x} - \frac{\Delta t}{2} \left( \mathbf{f}(t_n, \mathbf{u}^{n}) +  \mathbf{f}(t_{n+1}, \mathbf{x}) \right) - \mathbf{u}^{n}  = 0
$$

in every step $n$, and define the approximate solution at the next step as $\mathbf{u}^{n+1} = \boldsymbol{\alpha}$.

<div class="alert alert-success">
    
**Exercise 3:** Complete the function `crank_nicolson_sys` which implements the Crank-Nicolson method to compute the approximations $\mathbf{u}^{n} \approx \mathbf{u}(t_n)$ for $n = 0, 1, 2, \dots, N$.

*Hint:* You can copy all the code of the `backward_euler_sys` function from above, and will just have to adapt the definition of the function `g`, which represents $\mathbf{g}_n(\mathbf{x})$, from the backward Euler to the Crank-Nicolson method.
</div>

In [ ]:
def crank_nicolson_sys(f, u_0, T, N):
    ### BEGIN SOLUTION
    dt = T / N
    u = np.zeros((N + 1, len(u_0)))
    u[0] = u_0
    for n in range(N):
        # Function for which we need to find a zero
        def g(x):
            return x - dt / 2 * (f(n * dt, u[n]) + f((n + 1) * dt, x)) - u[n]
        # Method which finds the zero of the function g
        u[n + 1] = sp.optimize.root(g, u[n], method="hybr").x
    ### END SOLUTION
    return u

<div class="alert alert-success">
    
**Exercise 4:** For the same system as in the previous exercises, plot the approximate solutions from the trapzoidal methods for $N=200$ time steps. How well do the methods approximate the exact solution?
</div>

In [ ]:
### BEGIN SOLUTION
u_crank_nicolson = crank_nicolson_sys(f, u_0, T, N)
plot_solution(t, u_crank_nicolson)

# The approximate solution with the Crank-Nicolson method approximately
# coincides with the exact solution, as it stays on the unit circle.

### END SOLUTION

<hr style="clear:both">

### Electrical circuit

We want to study an electrical circuit, which you can visualize by running the cell below.

In [ ]:
fig, ax = plt.subplots(); ax.axis("equal"); ax.axis("off"); ax.plot([0.4, 1, 1], [-1, -1, -0.1], linewidth=4, color="black"); ax.plot([1, 1, 0.4], [0.1, 1, 1], linewidth=4, color="black"); ax.plot([-0.4, -1, -1], [1, 1, 0.1], linewidth=4, color="black"); ax.plot([-1, -1, -0.4], [-0.1, -1, -1], linewidth=4, color="black"); ax.plot([0.8, 1.2], [-0.1, -0.1], linewidth=4, color="black"); ax.plot([0.8, 1.2], [0.1, 0.1], linewidth=4, color="black"); ax.text(0.65, 0.25, "$C$", fontsize=30, horizontalalignment="center", verticalalignment="center"); ax.plot([-0.85, -1.15], [-0.1, -0.1], linewidth=4, color="black"); ax.plot([-0.7, -1.3], [0.1, 0.1], linewidth=4, color="black"); ax.text(-0.7, 0.3, "$+$", fontsize=30, horizontalalignment="center", verticalalignment="center"); ax.text(-0.7, -0.2, "$-$", fontsize=30, horizontalalignment="center", verticalalignment="center"); ax.add_patch(mpl.patches.Arc((-0.3, 1), 0.2, 0.4, theta1=0, theta2=180, linewidth=4)); ax.add_patch(mpl.patches.Arc((-0.1, 1), 0.2, 0.4, theta1=0, theta2=180, linewidth=4)); ax.add_patch(mpl.patches.Arc((0.1, 1), 0.2, 0.4, theta1=0, theta2=180, linewidth=4)); ax.add_patch(mpl.patches.Arc((0.3, 1), 0.2, 0.4, theta1=0, theta2=180, linewidth=4)); ax.text(0, 0.75, "$L$", fontsize=30, horizontalalignment="center", verticalalignment="center"); ax.plot([-0.4, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 0.4], [-1, -0.85, -1.15, -0.85, -1.15, -0.85, -1.15, -0.85, -1.15, -1], linewidth=4, color="black"); ax.text(0, -0.65, "$R$", fontsize=30, horizontalalignment="center", verticalalignment="center"); plt.show()

The circuit satisfies the following equation for the voltage $v$ around the capacitor:

$$
LC \frac{\mathrm{d}^2 v(t)}{\mathrm{d}t^2} + RC \frac{\mathrm{d} v(t)}{\mathrm{d}t} + v(t) = F
$$

where $L$ is the inductance of the coil, $C$ is the capacity of the capacitor, $R$ the resistance
and $F$ the motor force of the generator, which we assume to be constant in time. To apply the methods from the course, we will need to transform this second order equation into a system of first order equations.

<div class="alert alert-success">
    
**Exercise 5 (Theoretical):** Introduce a new unknown $w$ such as $w(t)=\frac{\mathrm{d} v(t)}{\mathrm{d}t}$. Use this relation to rewrite the equation above as a linear system of first order equations

$$
\frac{\mathrm{d} \mathbf{u}(t)}{\mathrm{d}t} = A \mathbf{u}(t) + \mathbf{b}(t) = \mathbf{f}(t, \mathbf{u}(t))
$$
where $\mathbf{u}(t) = ( w(t), v(t) )^\top$ and $A$ is a $2 \times 2$ matrix.
</div>

=== BEGIN MARK SCHEME ===

Using the relation $w(t)=\frac{\mathrm{d} v(t)}{\mathrm{d}t}$, we have $\frac{\mathrm{d} w(t)}{\mathrm{d}t} = \frac{\mathrm{d}^2 v(t)}{\mathrm{d}t^2}$ and therefore the second order ordinary differential equation can be written as

$$
LC \frac{\mathrm{d} w(t)}{\mathrm{d}t} + RC w(t) + v(t) = F,
$$

which leads to the system of first order equations

$$
  \begin{cases}
    \frac{\mathrm{d} w(t)}{\mathrm{d}t} = \frac{1}{LC}\,F - \frac{R}{L}\,w(t) - \frac{1}{LC}\,v(t),\\[8pt]
    \frac{\mathrm{d} v(t)}{\mathrm{d}t} = w(t)
  \end{cases}
$$

We can rewrite it in matrix form

$$
\frac{\mathrm{d}}{\mathrm{d}t}
\begin{pmatrix}
      w(t) \\ v(t)
\end{pmatrix}
=
\begin{pmatrix}
      % -\frac{R}{L} & -\frac{1}{LC} \\
      -R/L & -1/(LC) \\
      1 & 0 
\end{pmatrix}
\begin{pmatrix}
      w(t) \\ v(t)
\end{pmatrix}
+
\begin{pmatrix}
      F/(LC) \\ 0
\end{pmatrix}
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 6 (Theoretical):** We want to appoximate the solution of this system using the forward Euler method. Give the general stability criterion for the forward Euler method applied to linear systems. Then, compute the condition on $\Delta t$ for the forward Euler method to be stable for the system at hand, for the values $L=0.01$, $C=10$ and $R=0.1$.

*Hint:* Compute the eigenvalues of the matrix $A$ found in the previous exercise and apply Lemma 6.4 from the lecture notes. You can use Python for this.
</div>

=== BEGIN MARK SCHEME ===

For the forward Euler method, Lemma 6.4 from the lecture notes gives us the following stability criterion:

$$
|1 + \Delta t \lambda_i(A) | < 1
$$
where $\lambda_i(A)$, $i=1,2$ are the eigenvalues of the matrix $A$. We notice here that this
criterion applies only if the eigenvalues of the matrix are real and negative.
With the given values, we have

$$
A=
\left(\begin{array}{cc}
    -10 & -10 \\
    1 & 0 
  \end{array} \right),
$$

whose eigenvalues are $\lambda_1(A) = -8.873$ and $\lambda_2(A) = -1.127$, as can be computed with the following code:

```python
L = 0.01
C = 10
R = 0.1
F = 1.0
A = np.array([
    [-R / L, -1 / (L * C)],
    [1, 0]
])

eigenvalues = np.linalg.eigvals(A)
print("Eigenvalues of A:", eigenvalues)
```

The stability criterion is therefore satisfied when $\Delta t > 0$ is chosen such that 

$$
|1 - 8.873 \Delta t | < 1 \iff \Delta t < 2 / 8.873 = 0.225.
$$

This can be computed with the following code snippet:

```python
lambda_max = max(abs(eigenvalues))
print("Maximum magnitude of eigenvalues:", lambda_max)

dt_max = 2 / lambda_max
print("Maximum stable time step Δt:", dt_max)
```

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 7:** Consider the initial condition $\mathbf{u}_0 = (0,1)^\top$ and $F=0$. Using the forward Euler method (function `forward_euler_sys` which you have implemented above), compute an approximation of the solution of the system on the interval $[0,T]$ with $T=10$ and $N=43, 46,$ and $500$ sub-intervals, which corresponds to a time-step of $\Delta t= 0.233, 0.217,$ and $0.02$, respectively. Visualise the obtained approximations for $v(t)$ and comment on the obtained results.
</div>

In [ ]:
def f(t, u, R=0.1, L=0.01, C=10, F=0):
    # Returns the value of the right-hand side of the ODE
    ### BEGIN SOLUTION
    A = np.array([[-R/L, -1/(L*C)], [1, 0]])
    b = np.array([F/(L*C), 0])
    return A @ u + b
    ### END SOLUTION

### BEGIN SOLUTION
T = 10
u_0 = np.array([0, 1])

N_list = [43, 46, 500]
for N in N_list:
    u_forward_euler = forward_euler_sys(f, u_0, T, N)
    t = np.linspace(0, T, N + 1)
    plt.plot(t, u_forward_euler[:, 1], label=r"$N = {:d}$".format(N))

plt.ylabel(r"Forward Euler approximation $v^{n}$")
plt.xlabel(r"time $t$")
plt.legend()
plt.grid()
plt.show()

# We notice that the solution computed by the forward Euler method
# oscillates and does not approach 0 if we take N = 43 for which Δt = 0.233
# does not satisfy the stability condition. However, with N = 46, namely 
# Δt = 0.217 < 0.225, we get a numerical solution which approaches 0
# (oscillating). Finally, if we take N = 500, namely Δt = 0.02 which is way
# smaller than the minimum necessary value, we find a good approximation.
    
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 8:** Repeat the previous exercise using the backward Euler method (function `backward_euler_sys` which you have implemented above). Comment on the obtained results.
</div>

In [ ]:
### BEGIN SOLUTION
T = 10
u_0 = np.array([0, 1])

N_list = [43, 46, 500]
for N in N_list:
    u_backward_euler = backward_euler_sys(f, u_0, T, N)
    t = np.linspace(0, T, N + 1)
    plt.plot(t, u_backward_euler[:, 1], label=r"$N = {:d}$".format(N))

plt.ylabel(r"Backward Euler approximation $v^{n}$")
plt.xlabel(r"time $t$")
plt.legend()
plt.grid()
plt.show()

# Contrary to what was observed with the forward Euler method, we note that the
# backward Euler method gives stable results for the three considered time-steps.

### END SOLUTION

<hr style="clear:both">

## The end

Impeccable! You have finished the twelth exercise notebook. But there is one more.